In [1]:
import numpy as np
import pandas as pd
from senticnet5 import senticnet
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords,wordnet
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
import re
from scipy import sparse
stop_words = set(stopwords.words('english'))
wordnet_lemmatizer = WordNetLemmatizer()

In [2]:
df = pd.read_excel('hand8_k_random.xlsx')
print(len(df))

8501


In [3]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.multioutput import ClassifierChain
from skmultilearn.ensemble import RakelD

In [4]:
col_names = ['Joy','Sadness','Anger','Disgust','Admiration','Surprise','Interest','Fear']
X = df['Text']
Y = df[['Joy','Sadness','Anger','Disgust','Admiration','Surprise','Interest','Fear']]

In [5]:
vectorizer = TfidfVectorizer()
vectorizer.fit(df['Text'])
print(len(vectorizer.vocabulary_))

17402


In [6]:
kf = KFold(n_splits = 10)
kf.get_n_splits(X)
print(kf)

KFold(n_splits=10, random_state=None, shuffle=False)


In [7]:
def evaluation(score_list,predict_score_list):
    filter_corr = []
    exmatch = 0
    atleast1 = 0
    md1 = 0
    one_f = 0
    more_f = 0
    zero_f = 0
    sm = 0
    sdensity = 0
    hammval = 0
    test_len = len(predict_score_list[0])
    for j in range(test_len):
        cnt=0
        for i in range(8):
            hammval+=(score_list[i][j] ^ int(predict_score_list[i][j]))
            if(score_list[i][j]==1):
                cnt+=1
                sm+=1
        sdensity+=cnt/8
        if(cnt==0):
            zero_f+=1
        if(cnt==1):
            one_f+=1
        if(cnt>1):
            more_f+=1
        for i in range(8):
            mf = True
            if(int(predict_score_list[i][j])!=score_list[i][j]):
                mf=False
                break
        if(mf==True):
            exmatch+=1
            filter_corr.append(j)
        for i in range(8):
            if(int(predict_score_list[i][j])==score_list[i][j] and score_list[i][j]==1):
                atleast1+=1
                break
        mf = False
        for i in range(8):
            if(int(predict_score_list[i][j])==score_list[i][j] and score_list[i][j]==1):
                if(mf==True):
                    md1+=1
                    filter_corr.append(j)
                    break
                mf=True
    #print("Label Cardinality: "+ str(sm/test_len))
    #print("Label Density: "+ str(sdensity/test_len))
    print("Hamming Loss: "+str(hammval/(test_len*8)))
    hamlos = hammval/(test_len*8)
    print("Exact Prediction: "+str(exmatch/test_len))
    sub_accu = exmatch/test_len
    #print("At least one label predicted: "+str(atleast1/(test_len-zero_f)))
    #print("More than one label predicted: "+str(md1/more_f))
    tp_sum = 0
    fp_sum = 0
    fn_sum = 0
    macro_preci = 0
    macro_recall = 0
    macro_f1 = 0
    for i in range(len(score_list)):
        tmp = confusion_matrix(score_list[i],predict_score_list[i])
        tp_sum+=tmp[0][0]
        fp_sum+=tmp[0][1]
        fn_sum+=tmp[1][0]
        macro_preci_tmp=tmp[0][0]/(tmp[0][0]+tmp[0][1])
        macro_recall_tmp=tmp[0][0]/(tmp[0][0]+tmp[1][0])
        macro_f1 += ((2*macro_preci_tmp*macro_recall_tmp)/(macro_preci_tmp+macro_recall_tmp))
        macro_preci+=macro_preci_tmp
        macro_recall+=macro_recall_tmp
        #print(macro_f1)
    micro_preci = tp_sum/(tp_sum+fp_sum)
    micro_recall = tp_sum/(tp_sum+fn_sum)
    micro_f1 = (2*micro_preci*micro_recall)/(micro_preci+micro_recall)
    macro_preci/=8
    macro_recall/=8
    macro_f1/=8
    #print(micro_preci,micro_recall,micro_f1)
    #print(macro_preci,macro_recall,macro_f1)
    print("Macro F-Score: "+str(macro_f1))
    print("Micro F-Score: "+str(micro_f1))
    col_names = ['Joy','Sadness','Anger','Disgust','Admiration','Surprise','Interest','Fear']
    tmp = 0
    for i in range(len(score_list)):
        score = accuracy_score(score_list[i],predict_score_list[i]) 
        #print(col_names[i]+" accuracy: "+str(score))
        tmp += score
    print("Average Accuracy: " + str(tmp/8))
    avg_accu = tmp/8
    return (hamlos,sub_accu,macro_f1,micro_f1,avg_accu)

In [8]:
col_names = ['Joy','Sadness','Anger','Disgust','Admiration','Surprise','Interest','Fear']
hamm_score = []
subset_accu = []
macro_f1 = []
micro_f1 = []
avg_accu = []
cnt = 1
for train_index,test_index in kf.split(X):
    x_train,x_test = X.iloc[train_index],X.iloc[test_index]
    y_train,y_test = np.array(Y.iloc[train_index].values.tolist()),np.array(Y.iloc[test_index].values.tolist())
    print("k_fold validation: " + str(cnt))
    cnt+=1
    
    x_train = vectorizer.transform(x_train)
    x_test = vectorizer.transform(x_test)
    
    print(x_train.shape,x_test.shape)
    print(y_train.shape,y_test.shape)
    
    y_pred_val = []
    for i in range(8):
        print(i)
        classifier = RandomForestClassifier()
        tmp_y_train = y_train[:,i]
        classifier.fit(x_train,tmp_y_train)
        y_pred = classifier.predict(x_test)
        y_pred_val.append(y_pred)
    
    y_test_val = np.array(y_test)
    
    score_list = y_test_val.T.tolist()
    predict_score_list = y_pred_val
    
    ret = evaluation(score_list,predict_score_list)
    hamm_score.append(ret[0])
    subset_accu.append(ret[1])
    macro_f1.append(ret[2])
    micro_f1.append(ret[3])
    avg_accu.append(ret[4])
    print('\n')
print('Final Result: ')
print('Average Hamming Loss: '+str(sum(hamm_score)/len(hamm_score)))
print('Average Subset Accuracy: '+str(sum(subset_accu)/len(subset_accu)))
print('Average Macro F-score: '+str(sum(macro_f1)/len(macro_f1)))
print('Average Micro F-score: '+str(sum(micro_f1)/len(micro_f1)))
print('Average of Average Accuracy: '+str(sum(avg_accu)/len(avg_accu)))

k_fold validation: 1
(7650, 17402) (851, 17402)
(7650, 8) (851, 8)
0
1
2
3
4
5
6
7
Hamming Loss: 0.1605464159811986
Exact Prediction: 0.19741480611045828
Macro F-Score: 0.8942894655635346
Micro F-Score: 0.9080508118112224
Average Accuracy: 0.8394535840188014


k_fold validation: 2
(7651, 17402) (850, 17402)
(7651, 8) (850, 8)
0
1
2
3
4
5
6
7
Hamming Loss: 0.16897058823529412
Exact Prediction: 0.18470588235294116
Macro F-Score: 0.8920388474481976
Micro F-Score: 0.9032583985855014
Average Accuracy: 0.8310294117647058


k_fold validation: 3
(7651, 17402) (850, 17402)
(7651, 8) (850, 8)
0
1
2
3
4
5
6
7
Hamming Loss: 0.16779411764705882
Exact Prediction: 0.18235294117647058
Macro F-Score: 0.8916903666920724
Micro F-Score: 0.9035095137420719
Average Accuracy: 0.8322058823529412


k_fold validation: 4
(7651, 17402) (850, 17402)
(7651, 8) (850, 8)
0
1
2
3
4
5
6
7
Hamming Loss: 0.16558823529411765
Exact Prediction: 0.19647058823529412
Macro F-Score: 0.8918632878309152
Micro F-Score: 0.904930766

In [9]:
labels = [i+1for i in range(10)]
labels.append('average')
hamm_score.append(sum(hamm_score)/len(hamm_score))
subset_accu.append(sum(subset_accu)/len(subset_accu))
macro_f1.append(sum(macro_f1)/len(macro_f1))
micro_f1.append(sum(micro_f1)/len(micro_f1))
avg_accu.append(sum(avg_accu)/len(avg_accu))

In [10]:
df_rfc = pd.DataFrame(list(zip(labels,hamm_score,subset_accu,macro_f1,micro_f1,avg_accu)),
              columns = ['k-fold','Hamming loss','Subset accuracy','Macro F-score','Micro F-score','Average Accuracy'])
df_rfc

,k-fold,Hamming loss,Subset accuracy,Macro F-score,Micro F-score,Average Accuracy
0,1,0.160546,0.197415,0.894289,0.908051,0.839454
1,2,0.168971,0.184706,0.892039,0.903258,0.831029
2,3,0.167794,0.182353,0.891690,0.903510,0.832206
3,4,0.165588,0.196471,0.891863,0.904931,0.834412
4,5,0.163529,0.192941,0.894122,0.906460,0.836471
5,6,0.165147,0.185882,0.892652,0.905479,0.834853
6,7,0.161029,0.187059,0.895956,0.907462,0.838971
7,8,0.162206,0.215294,0.895002,0.906959,0.837794
8,9,0.166176,0.217647,0.891641,0.904221,0.833824
9,10,0.168824,0.161176,0.890521,0.903237,0.831176


In [11]:
col_names = ['Joy','Sadness','Anger','Disgust','Admiration','Surprise','Interest','Fear']
hamm_score = []
subset_accu = []
macro_f1 = []
micro_f1 = []
avg_accu = []
cnt = 1
for train_index,test_index in kf.split(X):
    x_train,x_test = X.iloc[train_index],X.iloc[test_index]
    y_train,y_test = np.array(Y.iloc[train_index].values.tolist()),np.array(Y.iloc[test_index].values.tolist())
    print("k_fold validation: " + str(cnt))
    cnt+=1
    
    x_train = vectorizer.transform(x_train)
    x_test = vectorizer.transform(x_test)
    
    print(x_train.shape,x_test.shape)
    print(y_train.shape,y_test.shape)
    
    y_pred_val = []
    for i in range(8):
        print(i)
        classifier = SVC()
        tmp_y_train = y_train[:,i]
        classifier.fit(x_train,tmp_y_train)
        y_pred = classifier.predict(x_test)
        y_pred_val.append(y_pred)
    
    y_test_val = np.array(y_test)
    
    score_list = y_test_val.T.tolist()
    predict_score_list = y_pred_val
    
    ret = evaluation(score_list,predict_score_list)
    hamm_score.append(ret[0])
    subset_accu.append(ret[1])
    macro_f1.append(ret[2])
    micro_f1.append(ret[3])
    avg_accu.append(ret[4])
    print('\n')
print('Final Result: ')
print('Average Hamming Loss: '+str(sum(hamm_score)/len(hamm_score)))
print('Average Subset Accuracy: '+str(sum(subset_accu)/len(subset_accu)))
print('Average Macro F-score: '+str(sum(macro_f1)/len(macro_f1)))
print('Average Micro F-score: '+str(sum(micro_f1)/len(micro_f1)))
print('Average of Average Accuracy: '+str(sum(avg_accu)/len(avg_accu)))

k_fold validation: 1
(7650, 17402) (851, 17402)
(7650, 8) (851, 8)
0
1
2
3
4
5
6
7
Hamming Loss: 0.15716803760282022
Exact Prediction: 0.19506462984723855
Macro F-Score: 0.8951333978893801
Micro F-Score: 0.9095672751859365
Average Accuracy: 0.8428319623971798


k_fold validation: 2
(7651, 17402) (850, 17402)
(7651, 8) (850, 8)
0
1
2
3
4
5
6
7
Hamming Loss: 0.16647058823529412
Exact Prediction: 0.19411764705882353
Macro F-Score: 0.8924726896315498
Micro F-Score: 0.9044886938913264
Average Accuracy: 0.8335294117647059


k_fold validation: 3
(7651, 17402) (850, 17402)
(7651, 8) (850, 8)
0
1
2
3
4
5
6
7
Hamming Loss: 0.1675
Exact Prediction: 0.18235294117647058
Macro F-Score: 0.8908023015800226
Micro F-Score: 0.903253206489425
Average Accuracy: 0.8325


k_fold validation: 4
(7651, 17402) (850, 17402)
(7651, 8) (850, 8)
0
1
2
3
4
5
6
7
Hamming Loss: 0.16176470588235295
Exact Prediction: 0.2011764705882353
Macro F-Score: 0.893066983516237
Micro F-Score: 0.9067322367305408
Average Accuracy: 0

In [12]:
labels = [i+1for i in range(10)]
labels.append('average')
hamm_score.append(sum(hamm_score)/len(hamm_score))
subset_accu.append(sum(subset_accu)/len(subset_accu))
macro_f1.append(sum(macro_f1)/len(macro_f1))
micro_f1.append(sum(micro_f1)/len(micro_f1))
avg_accu.append(sum(avg_accu)/len(avg_accu))

In [13]:
df_svc = pd.DataFrame(list(zip(labels,hamm_score,subset_accu,macro_f1,micro_f1,avg_accu)),
              columns = ['k-fold','Hamming loss','Subset accuracy','Macro F-score','Micro F-score','Average Accuracy'])
df_svc

,k-fold,Hamming loss,Subset accuracy,Macro F-score,Micro F-score,Average Accuracy
0,1,0.157168,0.195065,0.895133,0.909567,0.842832
1,2,0.166471,0.194118,0.892473,0.904489,0.833529
2,3,0.167500,0.182353,0.890802,0.903253,0.832500
3,4,0.161765,0.201176,0.893067,0.906732,0.838235
4,5,0.165000,0.177647,0.891501,0.905188,0.835000
5,6,0.160147,0.214118,0.895468,0.908031,0.839853
6,7,0.160735,0.188235,0.894695,0.907208,0.839265
7,8,0.159706,0.216471,0.895930,0.908168,0.840294
8,9,0.163971,0.217647,0.892012,0.905340,0.836029
9,10,0.166618,0.183529,0.891105,0.904299,0.833382
